In [1]:
import awkward as ak
import numpy as np
import time
import coffea
print(coffea.__version__)
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
files = "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root"
#"root://cmseos.fnal.gov//store/user/hats/2021/JEC/RunIISummer19UL17NanoAOD_QCD_Pt-15to7000_NANOAODSIM_JMECustomTuples.root"
events = NanoEventsFactory.from_root(files, schemaclass=NanoAODSchema).events()

0.7.8


In [2]:
from coffea import processor, hist

class TriJetHists(processor.ProcessorABC):
    def __init__(self):
        dataset_cats = hist.Cat("dataset", "Dataset")
        mass_bins = hist.Bin("mass", "jet mass", 60, 0, 120)
        self._histos = processor.dict_accumulator({
        'mass1':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'mass2':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'mass3':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'genmass1':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'genmass2':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'genmass3':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'mass3_g':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'mass3_uds':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'mass3_c':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'mass3_b':hist.Hist(
            "Events", dataset_cats,
            mass_bins,
        ),
        'weights':hist.Hist(
            "Events",
            dataset_cats,
            hist.Bin("weights", "gen weights", 60, 0, 1),
        )})
    
    @property
    def accumulator(self):
        return self._histos
    
    # we will receive a NanoEvents instead of a coffea DataFrame
    def process(self, events):
        out = self.accumulator.identity()
        trijetevents = events[
            (ak.num(events.FatJet) >= 3) & (ak.num(events.GenJetAK8) >= 3)
        ]
        #get only jets matched with gen and store fakes
#         trijetevents = trijetevents[~ak.is_none(jet.matched_gen)]
#         fakes = trijetevents[ak.is_none(jet.matched_gen)]
        
        #get leading 3 jets
        jet1 = trijetevents.FatJet[:, 0]
        jet2 = trijetevents.FatJet[:, 1]
        jet3 = trijetevents.FatJet[:, 2]
        
        #calculate dphi_min
        dphi12 = np.abs(jet1.delta_phi(jet2))
        dphi13 = np.abs(jet1.delta_phi(jet3))
        dphi23 = np.abs(jet2.delta_phi(jet3))
        
        dphi_min = np.amin([dphi12, dphi13, dphi23], axis = 0)
        
        #do same for gen; jets might not be same order reco and gen
        genjet1 = jet1.matched_gen
        genjet2 = jet2.matched_gen
        genjet3 = jet3.matched_gen
        
        #do matching and fakes for each jet individually
        
        dphi12_gen = np.abs(genjet1.delta_phi(genjet2))
        dphi13_gen = np.abs(genjet1.delta_phi(genjet3))
        dphi23_gen = np.abs(genjet2.delta_phi(genjet3))
        
        dphimin_gen = np.amin([dphi12_gen, dphi13_gen, dphi23_gen], axis = 0)
        
        #apply dphi gen and reco selection
        trijetevents = trijetevents[(dphimin_gen > 1.0) & (dphi_min > 1.0)]
        jet1 = trijetevents.FatJet[:, 0]
        jet2 = trijetevents.FatJet[:, 1]
        jet3 = trijetevents.FatJet[:, 2]
        genjet1 = trijetevents.GenJetAK8[:, 0]
        genjet2 = trijetevents.GenJetAK8[:, 1]
        genjet3 = trijetevents.GenJetAK8[:, 2]

        #flavour --> 21 is gluon
        jet3_g = jet3[np.abs(genjet3.partonFlavour) == 21]
        jet3_uds = jet3[np.abs(genjet3.partonFlavour) < 4]
        jet3_c = jet3[np.abs(genjet3.partonFlavour) == 4]
        jet3_b = jet3[np.abs(genjet3.partonFlavour) == 5]
        
        out['mass1'].fill(
            dataset=events.metadata["dataset"],
            mass=jet1.mass,
            weight=trijetevents.Generator.weight
        )
        out['mass2'].fill(
            dataset=events.metadata["dataset"],
            mass=jet2.mass,
            weight=trijetevents.Generator.weight
        )
        out['mass3'].fill(
            dataset=events.metadata["dataset"],
            mass=jet3.mass,
            weight=trijetevents.Generator.weight
        )
        #NOTE --> need gen sd mass eventually --> recluster :( 
        out['genmass1'].fill(
            dataset=events.metadata["dataset"],
            mass=genjet1.mass,
            weight=trijetevents.Generator.weight
        )
        out['genmass2'].fill(
            dataset=events.metadata["dataset"],
            mass=genjet2.mass,
            weight=trijetevents.Generator.weight
        )
        out['genmass3'].fill(
            dataset=events.metadata["dataset"],
            mass=genjet3.mass,
            weight=trijetevents.Generator.weight
        )
        out['mass3_g'].fill(
            dataset=events.metadata["dataset"],
            mass=jet3_g.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 21]
        )
        out['mass3_uds'].fill(
            dataset=events.metadata["dataset"],
            mass=jet3_uds.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) < 4]
        )
        out['mass3_c'].fill(
            dataset=events.metadata["dataset"],
            mass=jet3_c.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 4]
        )
        out['mass3_b'].fill(
            dataset=events.metadata["dataset"],
            mass=jet3_b.mass,
            weight=trijetevents.Generator.weight[np.abs(genjet3.partonFlavour) == 5]
        )
        out['weights'].fill(
            dataset=events.metadata["dataset"],
            weights=trijetevents.Generator.weight,
        )                                          
        return out
    
    def postprocess(self, accumulator):
        return accumulator

In [3]:

samples = {
    "QCD": [files]
}

result = processor.run_uproot_job(
    samples,
    "Events",
    TriJetHists(),
    processor.iterative_executor,
    {"schema": NanoAODSchema},
)

Preprocessing:   0%|          | 0/1 [00:00<?, ?file/s]

Processing:   0%|          | 0/9 [00:00<?, ?chunk/s]

ValueError: ak.to_numpy cannot convert 'None' values to np.ma.MaskedArray unless the 'allow_missing' parameter is set to True

(https://github.com/scikit-hep/awkward-1.0/blob/1.5.0/src/awkward/operations/convert.py#L314)

Failed processing file: WorkItem(dataset='QCD', filename='root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root', treename='Events', entrystart=0, entrystop=103445, fileuuid=b'\xf6\x13I\x9c\xed\xdf\x11\xeb\x86Y\x80\x06\r\n\xbe\xef', usermeta={})

In [ ]:

%matplotlib inline

hist.plot1d(result['mass3_g'])

In [ ]:
hist.plot1d(result['mass3_uds'])

In [ ]:
hist.plot1d(result['mass3_b'])

In [ ]:
hist.plot1d(result['mass3_c'])

In [ ]:
hist.plot1d(result['mass3'])

In [ ]:
hist.plot1d(result['weights'])

In [ ]:
print(events.GenJetAK8.fields)

In [ ]:
print(events.FatJet.fields)

In [ ]:
events.GenJetAK8.fields

In [ ]:
#plot pt, eta, phi, fake rate, pt efficiency, gluon purity for difference selections with b tags for all jets 
#and each individual jet in the 3 jets